### Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Remove pre-existing logs

In [ ]:
# !rm -r /content/linear/ /content/attention/

### Import Libraries

In [ ]:
import logging
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import sys
import time
import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from torchvision.datasets import SVHN
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.utils.data.dataset import TensorDataset

### Add GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Choose FF Type and Dataset

In [ ]:
ff_type = 'linear'
# ff_type = 'attention'

# dataset = 'MNIST'
# dataset = 'FashionMNIST'
# dataset = 'CIFAR10'
# dataset = 'CIFAR100'
dataset = 'SVHN'

### Create Log Directories

In [ ]:
"""
if not os.path.exists(ff_type):
  os.mkdir(ff_type)
        
path = os.path.join(ff_type, dataset)
if not os.path.exists(path):
  os.mkdir(path)
"""

'\nif not os.path.exists(ff_type):\n  os.mkdir(ff_type)\n        \npath = os.path.join(ff_type, dataset)\nif not os.path.exists(path):\n  os.mkdir(path)\n'

### Define Nvidia SMI logging to be run as a background process

In [ ]:
"""
def start_logging(logger):
    print("******* nvidia-smi logging started *******")
    while True:
        nvidia_smi_entry = os.popen("nvidia-smi --query-gpu=timestamp,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.free,memory.total,power.draw --format=csv | sed -n '2 p'").read()
        logger.info(nvidia_smi_entry)
    print("******* nvidia-smi logging ended *******")
"""

'\ndef start_logging(logger):\n    print("******* nvidia-smi logging started *******")\n    while True:\n        nvidia_smi_entry = os.popen("nvidia-smi --query-gpu=timestamp,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.free,memory.total,power.draw --format=csv | sed -n \'2 p\'").read()\n        logger.info(nvidia_smi_entry)\n    print("******* nvidia-smi logging ended *******")\n'

### Setup Loggers

In [ ]:
'''
formatter = logging.Formatter('%(message)s')
def setup_logger(name, log_file, level=logging.INFO):
    """To setup as many loggers as you want"""

    handler = logging.FileHandler(log_file)
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    
    return logger
'''

'\nformatter = logging.Formatter(\'%(message)s\')\ndef setup_logger(name, log_file, level=logging.INFO):\n    """To setup as many loggers as you want"""\n\n    handler = logging.FileHandler(log_file)\n    handler.setFormatter(formatter)\n\n    logger = logging.getLogger(name)\n    logger.setLevel(level)\n    logger.addHandler(handler)\n    \n    return logger\n'

### Initialize Loggers

In [ ]:
"""
# nvidia smi logger

logger_nvidia_smi = setup_logger(path+"_logger_nvidia_smi", os.path.join(path, "nvidia_smi.csv"))
system_logger = multiprocessing.Process(target=start_logging, args=[logger_nvidia_smi])

# layer loggers

logger_layer_ff = setup_logger(path+"_logger_layer_ff", os.path.join(path, "layer_ff.csv"))
logger_layer_ff.info('layer_num,layertime_ms')

# e2e loggers

logger_e2e_ff = setup_logger(path+"_logger_e2e_ff", os.path.join(path, "e2e_ff.csv"))
logger_e2e_ff.info('e2etime_ms')

# linear ff loggers

logger_gpu_compute_linear_ff = setup_logger(path+"_logger_gpu_compute_linear_ff", os.path.join(path, "gpu_compute_linear_ff.csv"))
logger_gpu_compute_linear_ff.info('epoch,gpucomputetime_ms')

logger_epoch_linear_ff = setup_logger(path+"_logger_epoch_linear_ff", os.path.join(path, "epoch_linear_ff.csv"))
logger_epoch_linear_ff.info('epoch_num,epochtime_ms')

# attention ff loggers

logger_gpu_compute_attention_ff = setup_logger(path+"_logger_gpu_compute_attention_ff", os.path.join(path, "gpu_compute_attention_ff.csv"))
logger_gpu_compute_attention_ff.info('epoch,gpucomputetime_ms')

logger_epoch_attention_ff = setup_logger(path+"_logger_epoch_attention_ff", os.path.join(path, "epoch_attention_ff.csv"))
logger_epoch_attention_ff.info('epoch_num,epochtime_ms')
"""

'\n# nvidia smi logger\n\nlogger_nvidia_smi = setup_logger(path+"_logger_nvidia_smi", os.path.join(path, "nvidia_smi.csv"))\nsystem_logger = multiprocessing.Process(target=start_logging, args=[logger_nvidia_smi])\n\n# layer loggers\n\nlogger_layer_ff = setup_logger(path+"_logger_layer_ff", os.path.join(path, "layer_ff.csv"))\nlogger_layer_ff.info(\'layer_num,layertime_ms\')\n\n# e2e loggers\n\nlogger_e2e_ff = setup_logger(path+"_logger_e2e_ff", os.path.join(path, "e2e_ff.csv"))\nlogger_e2e_ff.info(\'e2etime_ms\')\n\n# linear ff loggers\n\nlogger_gpu_compute_linear_ff = setup_logger(path+"_logger_gpu_compute_linear_ff", os.path.join(path, "gpu_compute_linear_ff.csv"))\nlogger_gpu_compute_linear_ff.info(\'epoch,gpucomputetime_ms\')\n\nlogger_epoch_linear_ff = setup_logger(path+"_logger_epoch_linear_ff", os.path.join(path, "epoch_linear_ff.csv"))\nlogger_epoch_linear_ff.info(\'epoch_num,epochtime_ms\')\n\n# attention ff loggers\n\nlogger_gpu_compute_attention_ff = setup_logger(path+"_logg

### Initialize Timers

In [ ]:
"""
# layer ff timers

layer_ff_start = torch.cuda.Event(enable_timing=True)
layer_ff_end = torch.cuda.Event(enable_timing=True)

# e2e ff timers

e2e_ff_start = torch.cuda.Event(enable_timing=True)
e2e_ff_end = torch.cuda.Event(enable_timing=True)

# linear ff timers

gpu_compute_linear_ff_start = torch.cuda.Event(enable_timing=True)
gpu_compute_linear_ff_end = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_start = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_end = torch.cuda.Event(enable_timing=True)

# attention ff timers

gpu_compute_attention_ff_start = torch.cuda.Event(enable_timing=True)
gpu_compute_attention_ff_end = torch.cuda.Event(enable_timing=True)
epoch_attention_ff_start = torch.cuda.Event(enable_timing=True)
epoch_attention_ff_end = torch.cuda.Event(enable_timing=True)
e2e_attention_ff_start = torch.cuda.Event(enable_timing=True)
e2e_attention_ff_end = torch.cuda.Event(enable_timing=True)
"""

'\n# layer ff timers\n\nlayer_ff_start = torch.cuda.Event(enable_timing=True)\nlayer_ff_end = torch.cuda.Event(enable_timing=True)\n\n# e2e ff timers\n\ne2e_ff_start = torch.cuda.Event(enable_timing=True)\ne2e_ff_end = torch.cuda.Event(enable_timing=True)\n\n# linear ff timers\n\ngpu_compute_linear_ff_start = torch.cuda.Event(enable_timing=True)\ngpu_compute_linear_ff_end = torch.cuda.Event(enable_timing=True)\nepoch_linear_ff_start = torch.cuda.Event(enable_timing=True)\nepoch_linear_ff_end = torch.cuda.Event(enable_timing=True)\n\n# attention ff timers\n\ngpu_compute_attention_ff_start = torch.cuda.Event(enable_timing=True)\ngpu_compute_attention_ff_end = torch.cuda.Event(enable_timing=True)\nepoch_attention_ff_start = torch.cuda.Event(enable_timing=True)\nepoch_attention_ff_end = torch.cuda.Event(enable_timing=True)\ne2e_attention_ff_start = torch.cuda.Event(enable_timing=True)\ne2e_attention_ff_end = torch.cuda.Event(enable_timing=True)\n'

### Define Overlay

In [ ]:
def overlay_y_on_x(x, y, num_labels):
    """Replace the first num_label pixels of data [x] with one-hot-encoded label [y]
    """
    x_ = x.clone()
    x_[:, :num_labels] *= 0.0
    x_[range(x.shape[0]), y] = x.max()
    return x_

### Define Attention Layer

In [ ]:
class Attention_layer(nn.Module):

  def __init__(self, input_size: tuple, output_size: int, patch_size: int):
    """
      input_size: n_h, n_w, n_c
      output_size: int
      patch_size: int
    """
    super().__init__()
    self.input_size = input_size
    self.patch_size = patch_size
    self.output_size = output_size

    n_h, n_w, n_c = input_size
    patch_dim = n_h // patch_size
    num_patches = patch_dim ** 2 
    self.patch_embedding = nn.Conv2d(n_c, 64, kernel_size=patch_size, stride=patch_size, device=device)

    for param in self.patch_embedding.parameters():
      param.requires_grad = False

    self.self_attn = nn.MultiheadAttention(64, 8, device=device)
    self.fc1 = nn.Linear(64, output_size, device=device)
    self.lrelu = nn.ReLU() #nn.LeakyReLU()

  def __normalize(self, x):
    x_shape = x.shape
    x = x.reshape(x_shape[0], -1)
    x = x / (x.norm(2, 1, keepdim=True) + 1e-4)
    x = x.reshape(x_shape)
    return x

  def __reshape(self, x):
    x = x.reshape(x.size(0), self.input_size[2], self.input_size[0], self.input_size[1])
    return x

  def forward(self, x):
    x = self.__normalize(x) 
    x = self.__reshape(x) # (batch_size, channel_size, height, width)
    x = self.patch_embedding(x) # (batch_size, 64, patch_dim, patch_dim)
    x = x.flatten(2).transpose(1, 2) # (batch_size, num_patches, 64)

    x = x.transpose(0, 1)  # (num_patches, batch_size, 64)
    x, _ = self.self_attn(x, x, x)
    x = x.mean(dim=0) # (batch_size, 64)


    x = self.__normalize(x)
    x = self.fc1(x)

    x = self.lrelu(x)

    return x
  
  def train_ff(self, train_loader, epoch_range, batch_size, lr=0.03, threshold=2.0):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    next_pos = []
    next_neg = []
    next_label = []
    for e in epoch_range:
      # epoch_attention_ff_start.record() # start recording epoch time
      for (x_pos, x_neg, label) in train_loader:
        # gpu_compute_attention_ff_start.record() # start recording gpu compute time
        x_pos = x_pos.to(device)
        x_neg = x_neg.to(device)
        g_pos = self.forward(x_pos).pow(2).mean(dim=1)
        g_neg = self.forward(x_neg).pow(2).mean(dim=1)

        loss = torch.log(1 + torch.exp(
            torch.cat([
                threshold - g_pos,
                g_neg - threshold
            ])
        )).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        x_pos = x_pos.detach()
        x_neg = x_neg.detach()
        # gpu_compute_attention_ff_end.record() # end recording gpu compute time
        # torch.cuda.synchronize()
        # logger_gpu_compute_attention_ff.info(str(e)+","+str(gpu_compute_attention_ff_start.elapsed_time(gpu_compute_attention_ff_end))) # log gpu compute time
      # epoch_attention_ff_end.record() # end recording epoch time
      # torch.cuda.synchronize()
      # logger_epoch_attention_ff.info(str(e)+","+str(epoch_attention_ff_start.elapsed_time(epoch_attention_ff_end))) # log epoch time

    for (x_pos, x_neg, label) in train_loader:
      x_pos = x_pos.to(device)
      x_neg = x_neg.to(device)    
      x_pos_next = self.forward(x_pos).detach()
      x_neg_next = self.forward(x_neg).detach()
      next_pos.append(x_pos_next)
      next_neg.append(x_neg_next)
      next_label.append(label)
      x_pos = x_pos.detach()
      x_neg = x_neg.detach()
    
    next_pos = torch.cat(next_pos, dim=0)
    next_neg = torch.cat(next_neg, dim=0)
    next_label = torch.cat(next_label, dim=0)

    return DataLoader(TensorDataset(
        next_pos,
        next_neg,
        next_label
    ), batch_size=batch_size, shuffle=True)
    


### Define Linear Layer

In [ ]:
class Linear_layer(nn.Linear):

  def __init__(self, in_features, out_features,
                 bias=True, device=None, dtype=None):
    super().__init__(in_features, out_features, bias, device, dtype)
    self.relu = torch.nn.ReLU()

  def forward(self, x):
      x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4)
      return self.relu(
          torch.mm(x_direction, self.weight.T) +
          self.bias.unsqueeze(0))

  def train_ff(self, train_loader, epoch_range, batch_size, lr=0.03, threshold=2.0):
    
    optimizer = torch.optim.Adam(self.parameters(), lr=lr)
    next_pos = []
    next_neg = []
    next_label = []

    for i in epoch_range:
      # epoch_linear_ff_start.record() # start recording epoch time
      for (x_pos, x_neg, label) in train_loader:
        # gpu_compute_linear_ff_start.record() # start recording gpu compute time
        x_pos = x_pos.to(device)
        x_neg = x_neg.to(device)
        g_pos = self.forward(x_pos).pow(2).mean(1)
        g_neg = self.forward(x_neg).pow(2).mean(1)
        loss = torch.log(1 + torch.exp(torch.cat([
            -g_pos + threshold,
            g_neg - threshold]))).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        x_pos = x_pos.detach()
        x_neg = x_neg.detach()
        # gpu_compute_linear_ff_end.record() # end recording gpu compute time
        # torch.cuda.synchronize()
        # logger_gpu_compute_linear_ff.info(str(i)+","+str(gpu_compute_linear_ff_start.elapsed_time(gpu_compute_linear_ff_end))) # log gpu compute time
      # epoch_linear_ff_end.record() # end recording epoch time
      # torch.cuda.synchronize()
      # logger_epoch_linear_ff.info(str(i)+","+str(epoch_linear_ff_start.elapsed_time(epoch_linear_ff_end))) # log epoch time

    for (x_pos, x_neg, label) in train_loader:
      x_pos = x_pos.to(device)
      x_neg = x_neg.to(device)  
      x_pos_next = self.forward(x_pos).detach()
      x_neg_next = self.forward(x_neg).detach()
      next_pos.append(x_pos_next)
      next_neg.append(x_neg_next)
      next_label.append(label)
      x_pos = x_pos.detach()
      x_neg = x_neg.detach()
  
    next_pos = torch.cat(next_pos, dim=0)
    next_neg = torch.cat(next_neg, dim=0)
    next_label = torch.cat(next_label, dim=0)
    
    return DataLoader(TensorDataset(
        next_pos,
        next_neg,
        next_label
    ), batch_size=batch_size, shuffle=True)

### Define Net

In [ ]:
class Net(nn.Module):

  def __init__(self, layers, num_labels):
    super().__init__()
    self.layers = layers
    self.num_labels = num_labels

  def train_ff(self, train_loader, epochs=1000, **kwargs):
    cur_train_loader = train_loader
    batch_size = train_loader.batch_size
    for i, layer in enumerate(self.layers):
      
      is_large_batch = len(cur_train_loader) >= 5
      print(f"Training layer: {i+1} ... tqdm: {'loader' if not is_large_batch else 'epoch'}")

      cur_train_loader = tqdm(cur_train_loader) if not is_large_batch else cur_train_loader
      epoch_range = tqdm(range(epochs)) if is_large_batch else range(epochs)
      
      # layer_ff_start.record() # start recording layer train time
      cur_train_loader = layer.train_ff(cur_train_loader, epoch_range=epoch_range, batch_size=batch_size, **kwargs) # train layer
      # layer_ff_end.record() # end recording layer train time
      # torch.cuda.synchronize()
      # logger_layer_ff.info(str(i)+","+str(layer_ff_start.elapsed_time(layer_ff_end))) # log layer train time

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def predict_ff(self, data_loader):

    def predict(layers, x, num_labels):
      goodness_per_label = []
      for label in range(num_labels):
          h = overlay_y_on_x(x, label, num_labels)
          goodness = []
          for i, layer in enumerate(layers):
              h = layer(h)
              # if i==0:
                # continue
              goodness += [h.pow(2).mean(1)]
          goodness_per_label += [sum(goodness).unsqueeze(1)]
      goodness_per_label = torch.cat(goodness_per_label, 1)
      return goodness_per_label.argmax(1)
    
    preds = []
    labels = []
    for x, label in data_loader:
      x = x.to(device)
      preds.append(predict(self.layers, x, self.num_labels))
      labels.append(label)

    preds = torch.cat(preds, 0)
    labels = torch.cat(labels, 0)
    return preds.cpu(), labels.cpu()

  def predict_bp(self, data_loader):
    preds = []
    labels = []
    for input, label in data_loader:
      input = input.to(device)
      pred = self.forward(input)
      preds.append(pred.argmax(1))
      labels.append(label)
    preds = torch.cat(preds, 0)
    labels = torch.cat(labels,0)
    return preds.cpu(), labels.cpu()

### Define MNIST Dataset

In [ ]:
def MNIST_dataset():
  transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])
  train_data = MNIST('./data/MNIST/', train=True,
              download=True,
              transform=transform)
  test_data = MNIST('./data/MNIST/', train=False,
              download=True,
              transform=transform)
  return train_data, test_data

### Define Fashion MNIST Dataset

In [ ]:
def FashionMNIST_dataset():

    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = FashionMNIST('./data/FashionMNIST', train=True,
              download=True,
              transform=transform)

    test_loader = FashionMNIST('./data/FashionMNIST', train=False,
              download=True,
              transform=transform)

    return train_loader, test_loader

### Define CIFAR10 Dataset

In [ ]:
def CIFAR10_dataset():

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.4914, 0.4822, 0.4465)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = CIFAR10('./data/CIFAR10', train=True,
              download=True,
              transform=transform)

    test_loader = CIFAR10('./data/CIFAR10', train=False,
              download=True,
              transform=transform)

    return train_loader, test_loader

### Define CIFAR100 Dataset

In [ ]:
def CIFAR100_dataset():

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = CIFAR100('./data/CIFAR100', train=True,
              download=True,
              transform=transform)

    test_loader = CIFAR100('./data/CIFAR100', train=False,
              download=True,
              transform=transform)

    return train_loader, test_loader

### Define SVHN Dataset

In [ ]:
def SVHN_dataset():

    transform = Compose([
        ToTensor(),
        Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = SVHN('./data/SVHN', split='train',
              download=True,
              transform=transform)

    test_loader = SVHN('./data/SVHN', split='test',
              download=True,
              transform=transform)

    return train_loader, test_loader

### Define Train and Test Sets

In [ ]:
def create_ff_train_dataset(train_loader, num_labels):
  pos_set = []
  neg_set = []
  label_set = []
  for input, label in tqdm(train_loader):
    x_pos = overlay_y_on_x(input, label, num_labels)
    rnd = torch.randperm(input.size(0))
    x_neg = overlay_y_on_x(input, label[rnd], num_labels)
    pos_set.append(x_pos)
    neg_set.append(x_neg)
    label_set.append(label)
  pos_set = torch.cat(pos_set, 0)
  neg_set = torch.cat(neg_set, 0)
  label_set = torch.cat(label_set, 0)
  return TensorDataset(pos_set, neg_set, label_set)

def create_ff_val_dataset(val_loader):
  inputs = []
  labels = []

  for input, label in tqdm(val_loader):
    inputs.append(input)
    labels.append(label)
  inputs = torch.cat(inputs, 0)
  labels = torch.cat(labels, 0)
  return TensorDataset(inputs, labels)

### Set Random Seed

In [ ]:
torch.manual_seed(1234)

### Download Dataset

In [ ]:
if dataset == 'MNIST':
  train_data, test_data = MNIST_dataset()
elif dataset == 'FashionMNIST':
  train_data, test_data = FashionMNIST_dataset()
elif dataset == 'CIFAR10': 
  train_data, test_data = CIFAR10_dataset()
elif dataset == 'CIFAR100':
  train_data, test_data = CIFAR100_dataset()
elif dataset == 'SVHN':
  train_data, test_data = SVHN_dataset()
else:
  print('Invalid dataset')
  exit()
print(train_data, test_data)

Using downloaded and verified file: ./data/SVHN/train_32x32.mat
Using downloaded and verified file: ./data/SVHN/test_32x32.mat
Dataset SVHN
    Number of datapoints: 73257
    Root location: ./data/SVHN
    Split: train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.198, 0.201, 0.197))
               Lambda()
           ) Dataset SVHN
    Number of datapoints: 26032
    Root location: ./data/SVHN
    Split: test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.198, 0.201, 0.197))
               Lambda()
           )


### Subsample Train and Test Sets

In [ ]:
num_train = 1257
num_test = 1257 #26032
subsample_train_indices = torch.randperm(len(train_data))[:num_train]
subsample_test_indices = torch.randperm(len(test_data))[:num_test]
print('# Train Samples:', len(subsample_train_indices), ', # Test Samples:', len(subsample_test_indices))
subsample_train_dataset = DataLoader(train_data, batch_size=1024, shuffle=False, sampler=SubsetRandomSampler(subsample_train_indices))
subsample_test_dataset = DataLoader(test_data, batch_size=1024, shuffle=False, sampler=SubsetRandomSampler(subsample_test_indices))

# Train Samples: 1257 , # Test Samples: 1257


### Create Train and Test Sets

In [ ]:
train_dataset = create_ff_train_dataset(subsample_train_dataset, 10)
train_dataset_for_accuracy = create_ff_val_dataset(subsample_train_dataset)
test_dataset = create_ff_val_dataset(subsample_test_dataset)

100%|██████████| 2/2 [00:00<00:00,  5.15it/s]


### Define Net

In [ ]:
if ff_type == 'linear':
  
  layers = [
    # Linear_layer(28*28, 2000, device=device),
    Linear_layer(32*32*3, 2000, device=device),
    Linear_layer(2000, 2000, device=device),
    Linear_layer(2000, 2000, device=device),
    Linear_layer(2000, 2000, device=device)
  ]

elif ff_type == 'attention':
  
    layers = [
    # Attention_layer((28, 28, 1), 2000, 4),
    Attention_layer((32, 32, 3), 2000, 4),
    Linear_layer(2000, 2000, device=device),
    Linear_layer(2000, 2000, device=device),
    Linear_layer(2000, 2000, device=device)
  ]

else:
  print('Invalid net type')
  exit()

net = Net(layers, 10)

### Start Nvidia SMI logging

In [ ]:
# system_logger.start()

### Train Net

In [ ]:
# print(len(DataLoader(train_dataset, batch_size=512, shuffle=True)))
# print(len(DataLoader(test_dataset, batch_size=512, shuffle=True)))

In [ ]:
# e2e_ff_start.record() # start recording e2e train time
net.train_ff(DataLoader(train_dataset, batch_size=512, shuffle=True), epochs=60, lr=0.02, threshold=15)
# e2e_ff_end.record() # end recording e2e train time
# torch.cuda.synchronize()
# logger_e2e_ff.info(str(e2e_ff_start.elapsed_time(e2e_ff_end))) # log e2e train time

Training layer: 1 ... tqdm: loader


100%|██████████| 3/3 [00:00<00:00, 66.05it/s]


Training layer: 2 ... tqdm: loader


100%|██████████| 3/3 [00:00<00:00, 106.42it/s]


Training layer: 3 ... tqdm: loader


100%|██████████| 3/3 [00:00<00:00, 112.85it/s]


Training layer: 4 ... tqdm: loader


100%|██████████| 3/3 [00:00<00:00, 119.25it/s]


### Check Performance of Net

In [ ]:
print('Dataset:', dataset)
print('# Train Samples:', len(subsample_train_indices), ', # Test Samples:', len(subsample_test_indices))
pred, true = net.predict_ff(DataLoader(train_dataset_for_accuracy, batch_size=512, shuffle=False))
print('Train Accuracy:', accuracy_score(true, pred))
pred, true = net.predict_ff(DataLoader(test_dataset, batch_size=512, shuffle=False))
print('Test Accuracy:',accuracy_score(true, pred))
# f1_score(true, pred, average=None)

Dataset: SVHN
# Train Samples: 1257 , # Test Samples: 1257
Train Accuracy: 0.07637231503579953
Test Accuracy: 0.07875894988066826


### Stop Nvidia SMI logging

In [ ]:
# system_logger.terminate()

### Create .zip files of logs

In [ ]:
# !zip -r /content/linear.zip /content/linear/
# !zip -r /content/attention.zip /content/attention/